In [1]:
#visualisation and work with dfs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, TimeDistributed, Flatten
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

%matplotlib inline

Using TensorFlow backend.


# Data preparations

In [2]:
#Load the data into Pandas dataframe
data2 = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
#data3 = pd.read_csv('../task3_data/task3_training.tsv', sep='\t')[['type', 'tweet']]

# Vectorization by keras

# Vectorization by fasttext

In [3]:
import fasttext

max_words = 10000
max_len = 50


data2 = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df = data2[:5000]

df_ft = df.iloc[np.random.permutation(len(df))]
df_ft['class'] = df_ft['class'].apply(lambda x: '__label__' + str(x))

df_train, df_test = train_test_split(df_ft , test_size=0.2)

df_train[['class', 'tweet']].to_csv('TRAIN.txt', header=False, index=False, sep="\t")
df_test[['class', 'tweet']].to_csv('TEST.txt', header=False, index=False, sep="\t")
#print(df_train, df_test)

graph = []
maxima, maxima_idx = 0, 0

# просто отрабатывает модель с классификацией, но я просто забираю потом отсюда вектора слов 
for epoch in range(15):
    model = fasttext.train_supervised(input='TRAIN.txt', epoch=epoch)
    All, Pos = 0, 0
    for (clas, tweet) in zip(df_test['class'], df_test['tweet']):
        All += 1
        if int(clas[-1]) == int(model.predict(tweet, k=2)[0][0][-1]): 
            #print(int(clas[-1]),int(model.predict(tweet, k=2)[0][0][-1]))
            Pos += 1
    graph.append(Pos/All)
    if Pos/All > maxima: maxima, maxima_idx = Pos/All, epoch

# Inplace vectors instead of words with FT

In [4]:
#Split into training and test data.
X_train = df_train['tweet']
X_test = df_test['tweet']
y_train = df_train['class']
y_test = df_test['class']

y_train = y_train.apply(lambda x: x[-1])
y_test = y_test.apply(lambda x: x[-1])

# it's special. we can't work with other format
X_train = np.array(X_train)
X_test = np.array(X_test)

#по сути на этом этапе можно слегка почистить составляющие твитов
for i in range(len(X_train)):
    sentence = X_train[i].split(' ')
    sentence = list(map(lambda x: model.get_word_vector(x), sentence))
    X_train[i] = sentence

for i in range(len(X_test)):
    sentence = X_test[i].split(' ')
    sentence = list(map(lambda x: model.get_word_vector(x), sentence))
    X_test[i] = sentence

maxima = 0
for ar in X_train:
    if len(ar)>maxima: maxima = len(ar)
for ar in X_test:
    if len(ar)>maxima: maxima = len(ar)
max_len = maxima + 1
X_train.shape

(4000,)

# Vectorization FastText_gensim

# Inplace vectors instead of words with FT_gensim

In [5]:
pd.DataFrame(X_train.tolist())

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.011430344, -0.000858719, 0.008840087, -0.0...","[0.012319109, 0.00915785, -0.009795009, 0.0055...","[0.011202505, 0.0071724392, -0.008483626, -0.0...","[0.036744773, 0.04987316, -0.016577838, -0.017...","[-0.0028241782, 0.0077870404, -0.0052717077, 0...","[-0.010973334, -0.014939071, 0.005818955, 0.01...","[-0.02589282, -0.04968359, 0.01679055, 0.01290...","[-0.022065645, -0.016441794, 0.011309001, 0.01...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,None,None,None,None,None,None,None,None,None,None
1,"[0.015027661, 0.018021118, 0.00040423515, -0.0...","[0.0047256765, 0.020385548, -0.004530614, -0.0...","[0.0012205226, 0.0019941365, 0.008941311, -0.0...","[0.0074302102, 0.006836756, 0.002737403, 0.001...","[-0.06511036, -0.10040281, 0.040169448, 0.0474...","[0.0072029945, 0.027822087, -0.014349167, -0.0...","[-0.03939214, -0.0695844, 0.024385992, 0.01661...","[0.002345221, -0.006869149, 0.0038548429, 0.00...","[-0.011019329, -0.011985397, -0.004696723, -0....","[-0.012432635, -0.00025335673, 0.008642172, 0....",...,"[0.02596093, 0.037638783, -0.0063346964, -0.02...","[0.00074419833, -0.0046492526, 0.007533915, 0....","[0.0028279433, -0.004363677, 0.007416248, -0.0...",None,None,None,None,None,None,None
2,"[-0.008845175, -0.022952251, 0.011543368, 0.00...","[-0.006350323, -0.015608624, 0.00251729, 0.011...","[-0.025360944, -0.029327085, 0.014437797, 0.01...","[0.002212279, 0.026430987, -0.010053046, -0.01...","[-0.005059674, -0.027975194, 0.0024831165, 0.0...","[-0.02672379, -0.050524455, 0.016743334, 0.013...","[-0.005644727, -0.024963247, 0.014319663, -0.0...","[-0.010647269, -0.028531017, 0.013301577, 0.00...","[0.014979405, 0.01971644, -0.015958196, -0.004...","[-0.017117789, -0.017930249, 0.014988156, 0.00...",...,None,None,None,None,None,None,None,None,None,None
3,"[-0.0062129046, 0.006366262, 0.0006097448, 0.0...","[0.022907825, 0.039254557, -0.0072166063, -0.0...","[-0.008525153, -0.009017156, 0.0055799657, -0....","[-0.003419058, 0.01020469, -0.003561513, 0.004...","[0.02596093, 0.037638783, -0.0063346964, -0.02...","[0.015812222, 0.026028898, -0.015760524, 0.002...","[0.030520875, 0.039942157, -0.019082308, -0.01...","[0.009778226, 0.0045343954, -0.002317614, 0.00...","[-0.005040966, -0.00686298, -0.00041837912, -0...","[5.2078707e-05, 0.004227434, -0.0020284965, -0...",...,None,None,None,None,None,None,None,None,None,None
4,"[-0.039810456, -0.06868611, 0.03728692, 0.0200...","[-0.0183677, -0.028766219, 0.0030596294, 0.014...","[-0.016084261, -0.031824008, 0.019782774, 0.01...","[-0.01381214, -0.026999671, 0.006919707, -0.00...","[-0.011430344, -0.000858719, 0.008840087, -0.0...","[-0.032895014, -0.054483265, 0.020839907, 0.02...","[0.0012205226, 0.0019941365, 0.008941311, -0.0...","[0.01058171, 0.01203719, -0.010230746, -0.0070...","[0.011581565, 0.03465495, -0.016451146, -0.016...","[-0.016386144, -0.013341472, 0.013759281, 0.00...",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,"[0.0024598192, 0.0043631694, 0.0021458631, -0....","[0.0089760935, 0.03243532, -0.011108233, -0.00...","[0.0073925145, 0.0033326594, -0.009096698, -0....","[-0.005663136, 0.008214361, -0.009253132, -0.0...","[0.007683079, -0.0024194568, -0.008872763, -0....","[0.02014223, 0.03806537, -0.011232877, -0.0145...","[0.0089760935, 0.03243532, -0.011108233, -0.00...","[0.01058171, 0.01203719, -0.010230746, -0.0070...","[0.017739853, 0.022794748, -0.009815619, -0.00...","[0.013836435, 0.024633054, -0.006187208, -0.01...",...,None,None,None,None,None,None,None,None,None,None
3996,"[0.0012205226, 0.0019941365, 0.008941311, -0.0...","[-0.0042367084, -0.020540148, 0.012642752, 0.0...","[0.031393617, 0.03785822, -0.014265586, -0.020...","[-0.07329542, -0.1201929, 0.047154423, 0.04342...","[-0.02589282, -0.04968359, 0.01679055, 0.01

In [6]:
#ars = [[1,2,3], [4,5,6,7], [9,1]]

def add_pads(ars, max_len):
#     lenn = 0
#     for ar in ars: 
#         if len(ar)>lenn: lenn = len(ar)

    for i in range(len(ars)):
        while len(ars[i]) < max_len: ars[i].insert(0,np.array([0 for i in range(100)]))
    return ars

X_train = add_pads(X_train, max_len)
X_test = add_pads(X_test, max_len)

In [7]:
X_train = X_train.tolist()
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0043540327, -0.012740399, 0.0009312754, -0...","[-0.013564588, -0.016544074, 0.004876098, 0.01...","[0.001043694, 0.016900722, -0.0033274477, -0.0...","[0.0089760935, 0.03243532, -0.011108233, -0.00...","[-0.018888012, -0.03216657, 0.0041524908, 0.00...","[0.025706355, 0.0443609, -0.01340977, -0.01665...","[-0.0037569036, -0.01885538, 0.014934133, 0.00...","[0.0027111415, 0.018022569, -0.009577237, -0.0..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.0018480176, -0.0059478874, 0.011692314, 0.0...","[-0.011019329, -0.011985397, -0.004696723, -0....","[-0.04592152, -0.07467532, 0.03978836, 0.02169...","[0.010018119, 0.022384891, -0.014281201, -0.01...","[-0.009317631, -0.014832063, 0.009850859, 0.00...","[0.0035715527, 0.0030309292, -0.004448453, 0.0...","[0.0012686602, -0.0109466305, 0.00721828, 0.01...","[0.02596093, 0.037638783, -0.0063346964, -0.02...","[0.00074419833, -0.0046492526, 0.007533915, 0....","[0.0028279433, -0.004363677, 0.007416248, -0.0..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0.02308554, 0.026533144, -0.007592562, -0.013...","[0.00083508884, -0.006793678, -0.007240971, -0...","[0.0074302102, 0.006836756, 0.002737403, 0.001...","[0.0027893092, 0.0059788516, 0.00883639, -0.00...","[0.0008171404, -0.013286811, -0.0022164534, 0....","[7.185083e-05, 0.0014208198, -0.0005386385, -0...","[0.00083508884, -0.006793678, -0.007240971, -0...","[0.0074302102, 0.006836756, 0.002737403, 0.001...","[0.0016303455, 0.011667227, -0.00570926, -0.01...","[-0.0062959497, -0.025785392, 0.0044758567, 0...."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[-0.005040966, -0.00686298, -0.00041837912, -0...","[5.2078707e-05, 0.004227434, -0.0020284965, -0...","[-0.0077618845, -0.0002729038, -0.0017606764, ...","[0.026141329, 0.037716385, -0.009185047, -0.01...","[0.015450526, 0.042325813, -0.0039551985, -0.0...","[-0.0031636008, 0.00

In [8]:
X = np.array(X_train).reshape(len(X_train), len(X_train[1]), 100)

In [9]:
X.shape

(4000, 63, 100)

In [10]:
X_train = np.array(X_train)

# Configure LSTM

In [11]:
# configure network
#n_batch = len(X) # !!!!!
n_batch = 20
n_epoch = 10
n_neurons = 50

# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])


print(model.summary())

# fit network
for i in range(n_epoch):
    model.fit(X_train, y_train, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (20, 50)                  30200     
_________________________________________________________________
dense_1 (Dense)              (20, 1)                   51        
Total params: 30,251
Trainable params: 30,251
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
4000/4000 [==============================] - 3s 756us/step - loss: 0.1155 - acc: 0.9653
Epoch 1/1
4000/4000 [==============================] - 3s 640us/step - loss: 0.0409 - acc: 0.9865
Epoch 1/1
4000/4000 [==============================] - 3s 627us/step - loss: 0.0327 - acc: 0.9902
Epoch 1/1
4000/4000 [==============================] - 2s 603us/step - loss: 0.0312 - acc: 0.9915
Epoch 1/1
4000/4000 [==============================] - 2s 597us/step - loss: 0.0304 - acc: 0.9910
Epoch 1/1
4000/40

In [12]:
# аналогично X_train подготовлен X_test
X_test = X_test.tolist()
X_test = np.array(X_test).reshape(len(X_test), len(X_test[1]), 100)
X_test = np.array(X_test)
X_test.shape
type(X_test)

numpy.ndarray

In [72]:
# перекидываем веста в новую модель для размера батча 1
n_batch = 1

# re-define model
new_model = Sequential()
new_model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X_train.shape[1], X_train.shape[2]), stateful=True))
new_model.add(Dense(1, activation='softmax'))

# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)

# compile model
new_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

# Evaluate

In [73]:
predicts = []
N = 10
for i in range(N):
    testX, testy = np.array(X_train.tolist()[i]), np.array(y_test.tolist()[i])
    testX = testX.reshape(1, 63, 100)
    yhat = new_model.predict(testX)
    pred = 0 if yhat[0][0] < 0.5 else 1
    print(f'>Expected={y_test.tolist()[i]}, Predicted={pred}')
    predicts.append(pred)

>Expected=0, Predicted=1
>Expected=1, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1
>Expected=0, Predicted=1


In [74]:
from sklearn.metrics import classification_report
print(classification_report([int(i) for i in y_test[:N].to_list()], predicts))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.10      1.00      0.18         1

    accuracy                           0.10        10
   macro avg       0.05      0.50      0.09        10
weighted avg       0.01      0.10      0.02        10



/home/alexkay/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
